# 0) Imports and Installs 💻

In [61]:
!pip install yfinance

In [62]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date
from datetime import timedelta

# for graphs
import matplotlib.pyplot as plt

# Question 1: IPO Filings Web Scraping and Data Processing

What's the total sum ($m) of 2023 filings that happened on Fridays?

Re-use the [Code Snippet 1] example to get the data from web for this endpoint: https://stockanalysis.com/ipos/filings/ Convert the 'Filing Date' to datetime(), 'Shares Offered' to float64 (if '-' is encountered, populate with NaNs). Define a new field 'Avg_price' based on the "Price Range", which equals to NaN if no price is specified, to the price (if only one number is provided), or to the average of 2 prices (if a range is given). You may be inspired by the function extract_numbers() in [Code Snippet 4], or you can write your own function to "parse" a string. Define a column "Shares_offered_value", which equals to "Shares Offered" * "Avg_price" (when both columns are defined; otherwise, it's NaN)

Find the total sum in $m (millions of USD, closest INTEGER number) for all filings during 2023, which happened on Fridays (Date.dt.dayofweek()==4). You should see 32 records in total, 25 of it is not null.

(additional: you can read about S-1 IPO filing to understand the context)

## 1) [Code Snippet 1] IPOs data from Web


*   learn dataframes "stacking" (UNION ALL)

*   learn about type casting

*   generate new columns (simple and with function)

In [63]:
import pandas as pd
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/filings/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)

In [64]:
df_ipos = ipo_dfs[0]
df_ipos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325 entries, 0 to 324
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Filing Date     325 non-null    object
 1   Symbol          325 non-null    object
 2   Company Name    325 non-null    object
 3   Price Range     325 non-null    object
 4   Shares Offered  325 non-null    object
dtypes: object(5)
memory usage: 12.8+ KB


In [65]:
df_ipos

,Filing Date,Symbol,Company Name,Price Range,Shares Offered
0,"May 3, 2024",TBN,Tamboran Resources Corporation,-,-
1,"Apr 29, 2024",HWEC,"HW Electro Co., Ltd.",$3.00,3750000
2,"Apr 29, 2024",DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000
3,"Apr 26, 2024",EURK,Eureka Acquisition Corp,$10.00,5000000
4,"Apr 26, 2024",HDL,Super Hi International Holding Ltd.,-,-
...,...,...,...,...,...
320,"Jan 21, 2020",GOXS,"Goxus, Inc.",$8.00 - $10.00,1500000
321,"Jan 21, 2020",UTXO,"UTXO Acquisition, Inc.",$10.00,5000000
322,"Dec 9, 2019",LOHA,Loha Co. Ltd,$8.00 - $10.00,2500000
323,"Oct 4, 2019",ZGHB,China Eco-Materials Group Co. Limited,$4.00,4300000


In [66]:
# Specify the date format while converting to datetime
df_ipos['Filing Date'] = pd.to_datetime(df_ipos['Filing Date'], format="%b %d, %Y")

# Handle errors by coercing invalid values to NaT (Not a Time)
df_ipos['Shares Offered'] = pd.to_numeric(df_ipos['Shares Offered'], errors='coerce')

In [67]:
# import re
# import time
# type_a_str='6.50'
# type_b_str='$4.00'
# type_c_str='-'
# def extract_mean(text:str):

#   arr = []
#   if '-' in text:
#     arr = text.split('-')
#   elif "$" in text:
#     arr.append(text)
#   arr_out=[]
#   for a in arr:
#     arr_out.append(float(a.replace('$','').strip()) if not a=='' else 0)



#   return np.mean(arr_out)
def extract_mean2(text:str):
  return np.mean([float(n_txt.strip()) if not  n_txt=='' else 0 for n_txt in (text.replace('$','').split('-'))])
# t1 = time.time()
# print(f"tipo a {type_a_str} mean {extract_mean(type_a_str)}")
# print(f"tipo a {type_b_str} mean {extract_mean(type_b_str)}")
# print(f"tipo a {type_c_str} mean {extract_mean(type_c_str)}")
# print( f'time {time.time()-t1}')
# t1 = time.time()
# print(f"tipo a {type_a_str} mean {extract_mean2(type_a_str)}")
# print(f"tipo a {type_b_str} mean {extract_mean2(type_b_str)}")
# print(f"tipo a {type_c_str} mean {extract_mean2(type_c_str)}")
# print( f'time {time.time()-t1}')

In [68]:
# type_a_str='6.50'
# type_b_str='$4.00'
# type_c_str='-'
# # print(re.search(r'$\d+\.\d{2}',type_b_str))
# a = type_b_str
# np.mean([float(n_txt.strip()) if not  n_txt=='' else 0 for n_txt in (a.replace('$','').split('-'))])

In [69]:
# t1 = time.time()
# ipos_filings['Avg_price']=ipos_filings['Price Range'].apply(lambda x:extract_mean(x))
# print( f'time {time.time()-t1}')
# t1 = time.time()
df_ipos['Avg_price']=df_ipos['Price Range'].apply(lambda x:extract_mean2(x))
# print( f'time {time.time()-t1}')


In [70]:
df_ipos

,Filing Date,Symbol,Company Name,Price Range,Shares Offered,Avg_price
0,2024-05-03,TBN,Tamboran Resources Corporation,-,NaN,0.00
1,2024-04-29,HWEC,"HW Electro Co., Ltd.",$3.00,3750000.0,3.00
2,2024-04-29,DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000.0,10.00
3,2024-04-26,EURK,Eureka Acquisition Corp,$10.00,5000000.0,10.00
4,2024-04-26,HDL,Super Hi International Holding Ltd.,-,NaN,0.00
...,...,...,...,...,...,...
320,2020-01-21,GOXS,"Goxus, Inc.",$8.00 - $10.00,1500000.0,9.00
321,2020-01-21,UTXO,"UTXO Acquisition, Inc.",$10.00,5000000.0,10.00
322,2019-12-09,LOHA,Loha Co. Ltd,$8.00 - $10.00,2500000.0,9.00
323,2019-10-04,ZGHB,China Eco-Materials Group Co. Limited,$4.00,4300000.0,4.00


In [71]:
df_ipos.to_excel('df_ipos.xlsx')

In [72]:
df_ipos['Shares_offered_value']=df_ipos['Shares Offered']*df_ipos['Avg_price']

In [73]:
# df_ipos

# **Answer Q1**



In [74]:
fillings_fridays=df_ipos.loc[(df_ipos['Filing Date'].dt.dayofweek==4) & (df_ipos['Filing Date'].dt.year==2023)]['Shares_offered_value'].sum()/1e6
print(f"The fillings during 2023 are {fillings_fridays:.0f}$m.")

The fillings during 2023 are 286$m.


# Question 2: IPOs "Fixed days hold" strategy

**Find the optimal number of days X (between 1 and 30), where 75% quantile growth is the highest?**

Reuse [Code Snippet 1] to retrieve the list of IPOs from 2023 and 2024 (from URLs: https://stockanalysis.com/ipos/2023/ and https://stockanalysis.com/ipos/2024/). Get all OHLCV daily prices for all stocks with an "IPO date" before March 1, 2024 ("< 2024-03-01") - 184 tickers (without 'RYZB'). Please remove 'RYZB', as it is no longer available on Yahoo Finance.

Sometimes you may need to adjust the symbol name (e.g., 'IBAC' on stockanalysis.com -> 'IBACU' on Yahoo Finance) to locate OHLCV prices for all stocks. Also, you can see the ticker changes using this link. Some of the tickers (like 'DYCQ' and 'LEGT') were on the market less than 30 days (11 and 21 days, respectively). Let's leave them in the dataset; it just means that you couldn't hold them for more days than they were listed.

Let's assume you managed to buy a new stock (listed on IPO) on the first day at the [Adj Close] price]. Your strategy is to hold for exactly X full days (where X is between 1 and 30) and sell at the "Adj. Close" price in X days (e.g., if X=1, you sell on the next day). Find X, when the 75% quantile growth (among 185 investments) is the highest.

HINTs:

You can generate 30 additional columns: growth_future_1d ... growth_future_30d, join that with the table of min_dates (first day when each stock has data on Yahoo Finance), and perform vector operations on the resulting dataset.
You can use the DataFrame.describe() function to get mean, min, max, 25-50-75% quantiles.
Additional:

You can also ensure that the mean and 50th percentile (median) investment returns are negative for most X values, implying a wager for a "lucky" investor who might be in the top 25%.
What's your recommendation: Do you suggest pursuing this strategy for an optimal X?

In [75]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/2023/"
response = requests.get(url, headers=headers)

ipos_2023 = pd.read_html(response.text)
df_ipos_2023 = ipos_2023[0]

url = "https://stockanalysis.com/ipos/2024/"
response = requests.get(url, headers=headers)

ipos_2024 = pd.read_html(response.text)
df_ipos_2024 = ipos_2024[0]

In [76]:
df_ipos_2023.isna().sum()

IPO Date        0
Symbol          0
Company Name    0
IPO Price       0
Current         0
Return          0
dtype: int64

In [77]:
df_ipos_2024.isna().sum()

IPO Date        0
Symbol          0
Company Name    0
IPO Price       0
Current         0
Return          0
dtype: int64

In [78]:
df_ipos_2023.head()

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,"Dec 27, 2023",IROH,Iron Horse Acquisitions Corp.,$10.00,$10.05,0.50%
1,"Dec 19, 2023",LGCB,Linkage Global Inc,$4.00,$2.91,-27.25%
2,"Dec 15, 2023",ZKH,ZKH Group Limited,$15.50,$12.95,-19.97%
3,"Dec 15, 2023",BAYA,Bayview Acquisition Corp,$10.00,$10.18,1.80%
4,"Dec 14, 2023",INHD,Inno Holdings Inc.,$4.00,$0.62,-84.45%


In [79]:
df_ipos_2024.head()

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,"May 1, 2024",VIK,Viking Holdings Ltd.,$24.00,$28.65,19.38%
1,"Apr 26, 2024",ZONE,"CleanCore Solutions, Inc.",$4.00,$3.15,-21.25%
2,"Apr 25, 2024",RBRK,"Rubrik, Inc.",$32.00,$33.77,5.53%
3,"Apr 25, 2024",LOAR,Loar Holdings Inc.,$28.00,$49.83,77.96%
4,"Apr 25, 2024",MRX,Marex Group plc,$19.00,$19.17,0.53%


In [80]:
df_ipos_2023['IPO Date']=pd.to_datetime(df_ipos_2023['IPO Date'],errors='raise',format ='mixed')
df_ipos_2024['IPO Date']=pd.to_datetime(df_ipos_2024['IPO Date'],errors='raise',format='mixed')

In [81]:
df_ipos_2023.head()

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,2023-12-27,IROH,Iron Horse Acquisitions Corp.,$10.00,$10.05,0.50%
1,2023-12-19,LGCB,Linkage Global Inc,$4.00,$2.91,-27.25%
2,2023-12-15,ZKH,ZKH Group Limited,$15.50,$12.95,-19.97%
3,2023-12-15,BAYA,Bayview Acquisition Corp,$10.00,$10.18,1.80%
4,2023-12-14,INHD,Inno Holdings Inc.,$4.00,$0.62,-84.45%


In [82]:
df_ipos_2024.head()

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,2024-05-01,VIK,Viking Holdings Ltd.,$24.00,$28.65,19.38%
1,2024-04-26,ZONE,"CleanCore Solutions, Inc.",$4.00,$3.15,-21.25%
2,2024-04-25,RBRK,"Rubrik, Inc.",$32.00,$33.77,5.53%
3,2024-04-25,LOAR,Loar Holdings Inc.,$28.00,$49.83,77.96%
4,2024-04-25,MRX,Marex Group plc,$19.00,$19.17,0.53%


In [83]:
df_ipos_2023=df_ipos_2023.loc[df_ipos_2023['IPO Date']<"2024-03-01"]
df_ipos_2024=df_ipos_2024.loc[df_ipos_2024['IPO Date']<"2024-03-01"]

In [84]:
df_ipos_2023

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,2023-12-27,IROH,Iron Horse Acquisitions Corp.,$10.00,$10.05,0.50%
1,2023-12-19,LGCB,Linkage Global Inc,$4.00,$2.91,-27.25%
2,2023-12-15,ZKH,ZKH Group Limited,$15.50,$12.95,-19.97%
3,2023-12-15,BAYA,Bayview Acquisition Corp,$10.00,$10.18,1.80%
4,2023-12-14,INHD,Inno Holdings Inc.,$4.00,$0.62,-84.45%
...,...,...,...,...,...,...
149,2023-01-25,QSG,QuantaSing Group Ltd,$12.50,$3.19,-74.48%
150,2023-01-20,CVKD,"Cadrenal Therapeutics, Inc.",$5.00,$0.48,-90.40%
151,2023-01-13,SKWD,"Skyward Specialty Insurance Group, Inc.",$15.00,$37.61,150.73%
152,2023-01-13,ISRL,Israel Acquisitions Corp,$10.00,$10.93,9.20%


In [85]:
stock_lst=list(set(pd.concat([df_ipos_2023['Symbol'],df_ipos_2024['Symbol']],axis=0)))
stock_lst.remove('RYZB')
stock_lst.remove('PTHR')
stock_lst.remove('JVSA')
stock_lst.remove('DYCQ')
stock_lst.remove('LEGT')
stock_lst.append('PTHRF')
stock_lst.append('JVSAU')
stock_lst.append('DYCQU')
stock_lst.append('LEGT-UN')
stock_lst

['LXEO',
 'BLAC',
 'TXO',
 'TBBB',
 'PRZO',
 'BMR',
 'TRNR',
 'SUGP',
 'KVYO',
 'NNAG',
 'HYAC',
 'MNR',
 'ARM',
 'CGON',
 'JL',
 'NETD',
 'SRM',
 'ATGL',
 'AHR',
 'KSPI',
 'PMEC',
 'GENK',
 'PAPL',
 'GSIW',
 'TCJH',
 'FORL',
 'SRFM',
 'ELWS',
 'SMXT',
 'SWIN',
 'INHD',
 'RR',
 'HLP',
 'CHRO',
 'BREA',
 'SKWD',
 'SYNX',
 'JNVR',
 'HKIT',
 'OAKU',
 'HG',
 'QSG',
 'MSS',
 'SGMT',
 'PXDT',
 'BTSG',
 'TURB',
 'MWG',
 'LSDI',
 'SYT',
 'BIRK',
 'ANSC',
 'HAO',
 'OMH',
 'FIHL',
 'CART',
 'CVKD',
 'YIBO',
 'BOF',
 'AFJK',
 'NWGL',
 'GLAC',
 'GNLX',
 'WLGS',
 'CWD',
 'PMNT',
 'TPET',
 'USGO',
 'APGE',
 'MGX',
 'HSHP',
 'CHSN',
 'SPPL',
 'GUTS',
 'VTMX',
 'MLYS',
 'NCL',
 'SHIM',
 'GODN',
 'TMTC',
 'SYRA',
 'MIRA',
 'WRNT',
 'BOWN',
 'ANRO',
 'KYTX',
 'KGS',
 'PSBD',
 'AESI',
 'GVH',
 'GDTC',
 'INTS',
 'SPGC',
 'ALCY',
 'VSME',
 'SDHC',
 'CLBR',
 'DDC',
 'SPKL',
 'MARX',
 'AACT',
 'LQR',
 'JYD',
 'UMAC',
 'CCTG',
 'GPAK',
 'GDHG',
 'SVV',
 'KVUE',
 'CAVA',
 'SXTP',
 'PWM',
 'GXAI',
 'CRGX',
 'AS

In [86]:
len(stock_lst)

184

In [87]:
end_dt = date(2024,3,1)
df_OHLC=yf.download(stock_lst,end=end_dt,progress=True)
df_OHLC

[*********************100%%**********************]  184 of 184 completed


Price      Adj Close                                                     \
Ticker          AACT       AESI    AFJK        AHR    AITR  AIXI   ALCY   
Date                                                                      
2009-12-29       NaN        NaN     NaN        NaN     NaN   NaN    NaN   
2009-12-30       NaN        NaN     NaN        NaN     NaN   NaN    NaN   
2009-12-31       NaN        NaN     NaN        NaN     NaN   NaN    NaN   
2010-01-04       NaN        NaN     NaN        NaN     NaN   NaN    NaN   
2010-01-05       NaN        NaN     NaN        NaN     NaN   NaN    NaN   
...              ...        ...     ...        ...     ...   ...    ...   
2024-02-23     10.55  18.320000  10.175  13.365355  10.240  1.87  10.55   
2024-02-26     10.54  18.330000  10.180  13.031221  10.240  1.90  10.56   
2024-02-27     10.55  19.809999  10.174  13.316218  10.242  2.43  10.56   
2024-02-28     10.48  19.299999  10.180  13.404666  10.240  2.06  10.54   
2024-02-29     10.53  18.860001  10.154  13.434148  10.240  1.98  10.55   

Price                            ...      Volume                      \
Ticker        ANL   ANRO   ANSC  ...        VHAI      VSME      VTMX   
Date                             ...                                   
2009-12-29    NaN    NaN    NaN  ...         NaN       NaN       NaN   
2009-12-30    NaN    NaN    NaN  ...         NaN       NaN       NaN   
2009-12-31    NaN    NaN    NaN  ...         NaN       NaN       NaN   
2010-01-04    NaN    NaN    NaN  ...         NaN       NaN       NaN   
2010-01-05    NaN    NaN    NaN  ...         NaN       NaN       NaN   
...           ...    ...    ...  ...         ...       ...       ...   
2024-02-23  7.151  13.72  10.14  ...         NaN  166900.0  125000.0   
2024-02-26  7.150  15.31  10.15  ...   2444600.0  326800.0   77300.0   
2024-02-27  7.150  14.36  10.15  ...  10196900.0  963200.0  114200.0   
2024-02-28  7.150  15.33  10.13  ...   1554300.0  393900.0  165400.0   
2024-02-29  7.150  15.40  10.14  ...   1853300.0  177900.0   90100.0   

Price                                                                        
Ticker          WBUY       WLGS      WRNT YGFGF     YIBO      ZJYL      ZKH  
Date                                                                         
2009-12-29       NaN        NaN       NaN   NaN      NaN       NaN      NaN  
2009-12-30       NaN        NaN       NaN   NaN      NaN       NaN      NaN  
2009-12-31       NaN        NaN       NaN   NaN      NaN       NaN      NaN  
2010-01-04       NaN        NaN       NaN   NaN      NaN       NaN      NaN  
2010-01-05       NaN        NaN       NaN   NaN      NaN       NaN      NaN  
...              ...        ...       ...   ...      ...       ...      ...  
2024-02-23  225200.0   107100.0  128200.0   0.0  35700.0  323200.0  54100.0  
2024-02-26  137400.0  1060100.0   38900.0   0.0  20400.0  578500.0  42700.0  
2024-02-27  161600.0   375400.0  147600.0   0.0  19300.0  218900.0  40000.0  
2024-02-28  182900.0   266700.0  187200.0   0.0   9700.0  200900.0  55200.0  
2024-02-29   88600.0   139100.0   62700.0   0.0  11400.0  120300.0  76600.0  

[3566 rows x 1104 columns]

In [88]:
df=df_OHLC.copy()
df=df['Adj Close']
df.tail(30)

Ticker,AACT,AESI,AFJK,AHR,AITR,AIXI,ALCY,ANL,ANRO,ANSC,...,VHAI,VSME,VTMX,WBUY,WLGS,WRNT,YGFGF,YIBO,ZJYL,ZKH
Date,,,,,,,,,,,,,,,,,,,,,
2024-01-18,10.465,16.587967,NaN,NaN,10.200,1.79,10.490,9.050,NaN,10.10,...,NaN,0.369,36.618332,0.399,0.700,0.350,1.530,NaN,5.2575,16.730000
2024-01-19,10.470,16.172773,NaN,NaN,10.210,1.61,10.490,8.989,NaN,10.11,...,NaN,0.330,37.434284,0.380,0.584,0.333,1.570,NaN,5.2500,16.100000
2024-01-22,10.480,16.933960,NaN,NaN,10.210,1.58,10.500,9.120,NaN,10.13,...,NaN,0.319,37.334778,0.395,0.590,0.330,1.420,NaN,5.3500,16.200001
2024-01-23,10.480,16.775791,10.160,NaN,10.210,1.69,10.600,9.120,NaN,10.13,...,NaN,0.328,37.205418,0.450,0.610,0.350,1.270,NaN,8.3450,16.809999
2024-01-24,10.480,16.627508,10.160,NaN,10.200,1.82,10.510,9.213,NaN,10.12,...,NaN,0.649,37.444237,0.460,0.600,0.348,1.250,NaN,10.5925,18.520000
2024-01-25,10.480,17.072359,10.170,NaN,10.200,1.88,10.560,9.213,NaN,10.12,...,NaN,0.487,38.568653,0.467,0.620,0.350,1.060,2.790,13.8055,19.040001
2024-01-26,10.490,17.240414,10.170,NaN,10.210,1.90,10.560,9.213,NaN,10.12,...,NaN,0.650,38.528854,0.445,0.660,0.345,0.858,2.720,8.9000,17.150000
2024-01-29,10.500,17.368925,10.180,NaN,10.210,1.90,10.520,9.130,NaN,10.12,...,NaN,0.599,38.260185,0.464,0.659,0.344,0.685,2.600,9.1495,17.750000
2024-01-30,10.490,17.527094,10.170,NaN,10.200,1.94,10.525,9.130,NaN,10.13,...,NaN,0.478,37.941769,0.428,0.790,0.330,0.731,2.660,10.5200,18.120001


In [89]:
df_tmp = pd.DataFrame(columns=[f'growth_future_{d}d' for d in range(1,31)],index=df.columns)


# df_tmp

In [90]:
df_tmp

,growth_future_1d,growth_future_2d,growth_future_3d,growth_future_4d,growth_future_5d,growth_future_6d,growth_future_7d,growth_future_8d,growth_future_9d,growth_future_10d,...,growth_future_21d,growth_future_22d,growth_future_23d,growth_future_24d,growth_future_25d,growth_future_26d,growth_future_27d,growth_future_28d,growth_future_29d,growth_future_30d
Ticker,,,,,,,,,,,,,,,,,,,,,
AACT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AESI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AFJK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AHR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AITR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WRNT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
YGFGF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
YIBO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [91]:
stock_lst

['LXEO',
 'BLAC',
 'TXO',
 'TBBB',
 'PRZO',
 'BMR',
 'TRNR',
 'SUGP',
 'KVYO',
 'NNAG',
 'HYAC',
 'MNR',
 'ARM',
 'CGON',
 'JL',
 'NETD',
 'SRM',
 'ATGL',
 'AHR',
 'KSPI',
 'PMEC',
 'GENK',
 'PAPL',
 'GSIW',
 'TCJH',
 'FORL',
 'SRFM',
 'ELWS',
 'SMXT',
 'SWIN',
 'INHD',
 'RR',
 'HLP',
 'CHRO',
 'BREA',
 'SKWD',
 'SYNX',
 'JNVR',
 'HKIT',
 'OAKU',
 'HG',
 'QSG',
 'MSS',
 'SGMT',
 'PXDT',
 'BTSG',
 'TURB',
 'MWG',
 'LSDI',
 'SYT',
 'BIRK',
 'ANSC',
 'HAO',
 'OMH',
 'FIHL',
 'CART',
 'CVKD',
 'YIBO',
 'BOF',
 'AFJK',
 'NWGL',
 'GLAC',
 'GNLX',
 'WLGS',
 'CWD',
 'PMNT',
 'TPET',
 'USGO',
 'APGE',
 'MGX',
 'HSHP',
 'CHSN',
 'SPPL',
 'GUTS',
 'VTMX',
 'MLYS',
 'NCL',
 'SHIM',
 'GODN',
 'TMTC',
 'SYRA',
 'MIRA',
 'WRNT',
 'BOWN',
 'ANRO',
 'KYTX',
 'KGS',
 'PSBD',
 'AESI',
 'GVH',
 'GDTC',
 'INTS',
 'SPGC',
 'ALCY',
 'VSME',
 'SDHC',
 'CLBR',
 'DDC',
 'SPKL',
 'MARX',
 'AACT',
 'LQR',
 'JYD',
 'UMAC',
 'CCTG',
 'GPAK',
 'GDHG',
 'SVV',
 'KVUE',
 'CAVA',
 'SXTP',
 'PWM',
 'GXAI',
 'CRGX',
 'AS

Calculate the future growth using a function

In [92]:
def calc_growth_futre_xd(qty_days:int,stock:str,df_stocks:pd.DataFrame)->float:
  df_temp2=df.loc[df[stock].isna()==False,stock].copy()
  try:
    return (df_temp2.shift(-qty_days)/df_temp2).dropna().iloc[0]
  except Exception as e :
    return np.NaN

stock_lst.sort()

t1= time.time()
for ticket in stock_lst:
  for d in range(1,31):
    df_tmp.loc[ticket,f"growth_future_{d}d"]=calc_growth_futre_xd(d,ticket,df)
for c in df_tmp.columns:
  df_tmp[c]=  pd.to_numeric(df_tmp[c],errors='coerce')
# df_tmp.info()
# df_tmp.describe()
print(f"time enlapsed {time.time()-t1:.2f}")
# df_tmp.loc['AESI'][f"growth_future_{1}d"]=1
df_tmp

time enlapsed 6.37


,growth_future_1d,growth_future_2d,growth_future_3d,growth_future_4d,growth_future_5d,growth_future_6d,growth_future_7d,growth_future_8d,growth_future_9d,growth_future_10d,...,growth_future_21d,growth_future_22d,growth_future_23d,growth_future_24d,growth_future_25d,growth_future_26d,growth_future_27d,growth_future_28d,growth_future_29d,growth_future_30d
Ticker,,,,,,,,,,,,,,,,,,,,,
AACT,0.999011,1.000000,1.000000,1.000989,1.000989,1.000989,1.002967,1.002967,1.003956,1.004946,...,1.005539,1.004946,1.004946,1.004946,1.005935,1.005935,1.005935,1.007913,1.007913,1.007913
AESI,0.973451,0.943363,0.979941,0.946902,0.951032,0.939823,0.949852,0.965192,0.968141,0.943953,...,1.023009,1.049557,1.050737,1.058997,1.076106,1.074926,1.057817,1.066076,1.056637,1.072566
AFJK,1.000000,1.000984,1.000984,1.001969,1.000984,1.001969,1.000984,1.000984,1.000492,1.000984,...,1.001969,1.001476,1.001969,1.001378,1.001969,0.999409,NaN,NaN,NaN,NaN
AHR,0.987897,0.992436,0.994705,1.009834,1.040847,1.031770,1.049168,1.014372,1.013616,1.024962,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AITR,1.000986,1.003945,1.003945,1.004043,1.004931,1.004931,1.004931,1.005424,1.004931,1.005917,...,1.006903,1.006903,1.006903,1.007890,1.008876,1.007890,1.007692,1.007692,1.008383,1.008876
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WRNT,0.788372,0.572093,0.508139,0.441860,0.372093,0.402326,0.369767,0.365116,0.318605,0.313953,...,0.241860,0.223256,0.226744,0.209302,0.244186,0.217674,0.209302,0.209302,0.213953,0.225814
YGFGF,0.975064,0.976864,0.997686,1.002571,0.940874,0.866324,0.843188,0.843188,0.832905,0.858612,...,0.950386,0.904884,1.002571,0.866324,0.868895,0.750643,0.758355,0.754499,0.830334,0.943445
YIBO,0.974910,0.931900,0.953405,0.867384,0.821505,0.737993,0.727599,0.788530,0.770609,0.770609,...,0.931900,0.982079,1.014337,0.982079,NaN,NaN,NaN,NaN,NaN,NaN


In [93]:
df_tmp.describe()

,growth_future_1d,growth_future_2d,growth_future_3d,growth_future_4d,growth_future_5d,growth_future_6d,growth_future_7d,growth_future_8d,growth_future_9d,growth_future_10d,...,growth_future_21d,growth_future_22d,growth_future_23d,growth_future_24d,growth_future_25d,growth_future_26d,growth_future_27d,growth_future_28d,growth_future_29d,growth_future_30d
count,184.000000,184.000000,183.000000,182.000000,182.000000,182.000000,181.000000,181.000000,179.000000,179.000000,...,166.000000,166.000000,166.000000,163.000000,161.000000,159.000000,158.000000,158.000000,156.000000,154.000000
mean,0.947024,0.938983,0.933140,0.923065,0.915571,0.908249,0.896999,0.896464,0.897562,0.894573,...,0.916390,0.908184,0.907936,0.905222,0.893395,0.899622,0.933650,0.939100,0.937817,0.939323
std,0.171487,0.205262,0.246487,0.262997,0.294080,0.308881,0.299111,0.321879,0.354863,0.367778,...,0.529817,0.487425,0.493780,0.502070,0.498251,0.544488,0.831942,0.833163,0.835309,0.828949
min,0.153569,0.108733,0.086641,0.094257,0.081118,0.087677,0.085261,0.090123,0.095616,0.096997,...,0.048326,0.048326,0.049361,0.048326,0.048326,0.045219,0.044529,0.046945,0.042803,0.040387
25%,0.921392,0.879575,0.862366,0.846173,0.792430,0.784618,0.790333,0.750000,0.731103,0.711589,...,0.625299,0.616880,0.616976,0.633517,0.607884,0.606017,0.602782,0.626498,0.610721,0.623529
50%,1.000000,1.000000,0.997959,0.995516,0.996965,0.976002,0.980443,0.981308,0.987915,0.978735,...,0.977389,0.985499,0.984442,0.977778,0.966194,0.977839,0.973983,0.973220,0.968098,0.984353
75%,1.012352,1.018409,1.009769,1.009530,1.009926,1.007620,1.006897,1.008815,1.011662,1.014198,...,1.011947,1.022487,1.020213,1.021537,1.016716,1.023933,1.021156,1.025557,1.024041,1.020630
max,1.362069,1.464015,2.380000,2.083710,2.262443,2.529870,2.173913,2.359740,2.751948,3.176087,...,4.500000,3.871041,3.846154,3.803394,3.427273,4.817886,9.056122,9.081632,9.265306,9.372449


In [94]:
# df_tmp.to_excel("df_tmp.xlsx")

In [95]:
quantil_75=df_tmp.describe().loc['75%']
quantil_75

growth_future_1d     1.012352
growth_future_2d     1.018409
growth_future_3d     1.009769
growth_future_4d     1.009530
growth_future_5d     1.009926
growth_future_6d     1.007620
growth_future_7d     1.006897
growth_future_8d     1.008815
growth_future_9d     1.011662
growth_future_10d    1.014198
growth_future_11d    1.013850
growth_future_12d    1.019350
growth_future_13d    1.020168
growth_future_14d    1.014986
growth_future_15d    1.014078
growth_future_16d    1.013298
growth_future_17d    1.010832
growth_future_18d    1.011820
growth_future_19d    1.014823
growth_future_20d    1.014142
growth_future_21d    1.011947
growth_future_22d    1.022487
growth_future_23d    1.020213
growth_future_24d    1.021537
growth_future_25d    1.016716
growth_future_26d    1.023933
growth_future_27d    1.021156
growth_future_28d    1.025557
growth_future_29d    1.024041
growth_future_30d    1.020630
Name: 75%, dtype: float64

# **Answer Q2**

In [96]:
days=f"{quantil_75[quantil_75.max()==quantil_75].index[0]}"
days=days.replace("growth_future_","").replace('d','')
print(f"the optimal number of days is {days}.")

the optimal number of days is 28.


# Question 3: Is Growth Concentrated in the Largest Stocks?
**Get the share of days (percentage as int) when Large Stocks outperform (growth_7d - growth over 7 periods back) the Largest stocks?**

Reuse [Code Snippet 5] to obtain OHLCV stats for 33 stocks for 10 full years of data (2014-01-01 to 2023-12-31). You'll need to download slightly more data (7 periods before 2014-01-01 to calculate the growth_7d for the first 6 days correctly):

`US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']`

`EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']`

`INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']`

`LARGEST_STOCKS = US_STOCKS + EU_STOCKS + INDIA_STOCKS`

Now let's add the top 12-22 stocks (as of end-April 2024):

`NEW_US = ['TSLA','WMT','XOM','UNH','MA','PG','JNJ','MRK','HD','COST','ORCL']`

`NEW_EU = ['PRX.AS','CDI.PA','AIR.PA','SU.PA','ETN','SNY','BUD','DTE.DE','ALV.DE','MDT','AI.PA','EL.PA']`

`NEW_INDIA = ['BAJFINANCE.NS','MARUTI.NS','HCLTECH.NS','TATAMOTORS.NS','SUNPHARMA.NS','ONGC.NS','ADANIENT.NS','ADANIENT.NS','NTPC.NS','KOTAKBANK.NS','TITAN.NS']`

`LARGE_STOCKS = NEW_EU + NEW_US + NEW_INDIA`

You should be able to obtain stats for 33 LARGEST STOCKS and 32 LARGE STOCKS (from the actual stats on Yahoo Finance)

Calculate growth_7d for every stock and every day. Get the average daily growth_7d for the LARGEST_STOCKS group vs. the LARGE_STOCKS group.

For example, for the first of data you should have:

| Date   |      ticker_category      |  growth_7d |
|----------|:-------------:|------:|
| 2014-01-01 |  LARGE | 1.011684 |
| 2014-01-01 |   LARGEST   |   1.011797 |


On that day, the LARGEST group was growing faster than LARGE one (new stocks).

Calculate the number of days when the LARGE GROUP (new smaller stocks) outperforms the LARGEST GROUP, divide it by the total number of trading days (which should be 2595 days), and convert it to a percentage (closest INTEGER value). For example, if you find that 1700 out of 2595 days meet this condition, it means that 1700/2595 = 0.655, or approximately 66% of days, the LARGE stocks were growing faster than the LARGEST ones. This suggests that you should consider extending your dataset with more stocks to seek higher growth.

HINT: you can use pandas.pivot_table() to "flatten" the table (LARGE and LARGEST growth_7d as columns)

In [97]:
US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']

EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']

INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']

LARGEST_STOCKS = US_STOCKS + EU_STOCKS + INDIA_STOCKS

In [98]:
NEW_US = ['TSLA','WMT','XOM','UNH','MA','PG','JNJ','MRK','HD','COST','ORCL']

NEW_EU = ['PRX.AS','CDI.PA','AIR.PA','SU.PA','ETN','SNY','BUD','DTE.DE','ALV.DE','MDT','AI.PA','EL.PA']

NEW_INDIA = ['BAJFINANCE.NS','MARUTI.NS','HCLTECH.NS','TATAMOTORS.NS','SUNPHARMA.NS','ONGC.NS','ADANIENT.NS','ADANIENT.NS','NTPC.NS','KOTAKBANK.NS','TITAN.NS']
LARGE_STOCKS = NEW_EU + NEW_US + NEW_INDIA

In [99]:
LARGEST_STOCKS.sort()
LARGE_STOCKS.sort()
print(f"The qty largest stocks {len(LARGEST_STOCKS)}")
print(f"The qty large stocks {len(LARGE_STOCKS)}")

The qty largest stocks 33
The qty large stocks 34


In [100]:
start_date=date(2014,1,1)
end_date=date(2023,12,31)
start_date=start_date-timedelta(days=12)
df_LARGEST_STOCK = yf.download(LARGEST_STOCKS,start_date,end_date,progress=True)['Adj Close']

[*********************100%%**********************]  33 of 33 completed


In [101]:
# start_date=date(2014,1,1)
# end_date=date(2024,4,30)
# start_date=start_date-timedelta(days=12)
df_LARGE_STOCK = yf.download(LARGE_STOCKS,start_date,end_date,progress=True)['Adj Close']

[*********************100%%**********************]  33 of 33 completed


In [102]:
# df_LARGE_STOCK.info()

In [103]:
df_LARGEST_STOCK['growth_7d']=(df_LARGEST_STOCK/df_LARGEST_STOCK.shift(7)).mean(axis=1)
df_LARGEST_STOCK['ticket_category']='LARGEST'
df_LARGEST_STOCK=df_LARGEST_STOCK['2014-01-01':]

In [104]:
df_LARGEST_STOCK

Ticker,AAPL,ACN,AMZN,ASML,AVGO,BHARTIARTL.NS,BRK-B,CDI.PA,GOOG,HDB,...,RELIANCE.NS,RMS.PA,SAP,SBIN.NS,SIE.DE,TCS.NS,TTE,V,growth_7d,ticket_category
Date,,,,,,,,,,,,,,,,,,,,,
2014-01-01,NaN,NaN,NaN,NaN,NaN,288.068939,NaN,NaN,NaN,NaN,...,379.456116,NaN,NaN,160.800293,NaN,872.926392,NaN,NaN,1.011797,LARGEST
2014-01-02,17.296658,67.617828,19.898500,83.166679,39.869339,280.350128,117.500000,108.007080,27.724083,15.844660,...,373.564423,237.023560,71.747688,158.755035,61.293823,878.480164,32.874443,51.427006,0.998189,LARGEST
2014-01-03,16.916719,67.842850,19.822001,82.878525,40.020779,280.904541,117.570000,108.086693,27.521841,15.844660,...,368.974854,237.974121,71.257607,156.664230,61.609619,900.857605,32.622696,51.461906,0.996691,LARGEST
2014-01-06,17.008966,67.126083,19.681499,81.176651,39.748169,280.904541,116.279999,105.937706,27.828691,16.173000,...,365.004456,233.266266,71.333656,153.794525,61.417664,907.911621,32.524185,51.152428,0.990353,LARGEST
2014-01-07,16.887329,67.942871,19.901501,80.492294,40.126797,282.908905,116.190002,105.062202,28.365179,16.158928,...,359.518372,229.916412,71.130852,151.357529,61.727268,894.351135,32.617226,51.543346,0.987984,LARGEST
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22,193.353287,351.798248,153.419998,749.656372,1117.222046,989.549988,356.470001,700.377502,142.720001,67.000000,...,2565.050049,1929.481323,152.699997,636.750000,163.427063,3806.362549,66.854424,257.949005,1.013788,LARGEST
2023-12-26,192.803986,350.785828,153.410004,759.767517,1127.089966,999.849976,356.829987,NaN,142.820007,66.720001,...,2578.050049,NaN,153.179993,638.049988,NaN,3778.043701,67.332932,258.677643,1.014900,LARGEST
2023-12-27,192.903839,351.172943,153.339996,761.112427,1121.394287,1021.200012,356.950012,695.922852,141.440002,67.040001,...,2586.850098,1917.951416,155.899994,648.549988,164.146576,3793.621338,67.538017,258.448059,1.013800,LARGEST


In [105]:
df_LARGE_STOCK['growth_7d']=(df_LARGE_STOCK/df_LARGE_STOCK.shift(7)).mean(axis=1)
df_LARGE_STOCK['ticket_category']='LARGE'
df_LARGE_STOCK=df_LARGE_STOCK['2014-01-01':]

In [106]:
df_LARGE_STOCK

Ticker,ADANIENT.NS,AI.PA,AIR.PA,ALV.DE,BAJFINANCE.NS,BUD,CDI.PA,COST,DTE.DE,EL.PA,...,SU.PA,SUNPHARMA.NS,TATAMOTORS.NS,TITAN.NS,TSLA,UNH,WMT,XOM,growth_7d,ticket_category
Date,,,,,,,,,,,,,,,,,,,,,
2014-01-01,37.967476,NaN,NaN,NaN,152.003906,NaN,NaN,NaN,NaN,NaN,...,NaN,533.223511,368.095612,220.363174,NaN,NaN,NaN,NaN,1.009288,LARGE
2014-01-02,36.074409,53.805912,48.513958,79.434074,151.897415,80.872658,108.007103,94.629112,8.384731,65.886162,...,48.639275,534.620789,365.543121,216.114395,10.006667,63.748562,21.187395,64.282150,0.999348,LARGE
2014-01-03,35.085484,54.125713,48.385269,79.279114,150.571243,81.021027,108.086700,94.211380,8.481506,66.352272,...,48.678337,540.071045,356.069519,217.116898,9.970667,64.201645,21.117586,64.127487,0.999633,LARGE
2014-01-06,35.608196,53.033070,47.553116,79.589035,150.179245,80.083832,105.937675,93.496529,8.426206,67.008278,...,48.897087,548.036438,360.241821,217.498810,9.800000,63.466454,20.999435,64.224159,0.990707,LARGE
2014-01-07,34.181324,52.206921,47.964905,80.642799,150.430923,81.598961,105.062172,93.062798,8.467680,67.413979,...,48.358040,553.579712,358.425629,215.827957,9.957333,65.407005,21.063873,65.132797,0.990937,LARGE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22,2808.350098,177.360001,137.034973,242.600006,7295.299805,63.560879,700.377502,654.652466,21.037939,178.186188,...,180.960007,1236.600830,724.700012,3627.350098,252.539993,518.266907,52.039433,100.971611,1.003809,LARGE
2023-12-26,2865.449951,NaN,NaN,NaN,7162.299805,63.679134,NaN,657.596313,NaN,NaN,...,NaN,1240.428955,719.549988,3656.699951,256.609985,517.988037,51.959702,101.199486,1.014441,LARGE
2023-12-27,2843.350098,176.820007,137.015320,241.699997,7232.799805,63.925491,695.922852,664.754272,20.912367,177.656738,...,181.899994,1245.350830,740.900024,3689.250000,261.440002,520.737183,52.448040,100.723907,1.003292,LARGE


In [107]:
df_LARGE_STOCK.reset_index(inplace=True)
df_LARGEST_STOCK.reset_index(inplace=True)


df=pd.concat([df_LARGE_STOCK[['Date','growth_7d','ticket_category']], df_LARGEST_STOCK[['Date','growth_7d','ticket_category']]])

In [108]:
df.sort_values(by=['Date','ticket_category'],inplace=True)

In [109]:
df.tail(20)

Ticker,Date,growth_7d,ticket_category
2585,2023-12-15,1.018701,LARGE
2585,2023-12-15,1.034450,LARGEST
2586,2023-12-18,1.020206,LARGE
2586,2023-12-18,1.033641,LARGEST
2587,2023-12-19,1.020578,LARGE
2587,2023-12-19,1.030489,LARGEST
2588,2023-12-20,0.999913,LARGE
2588,2023-12-20,1.016704,LARGEST
2589,2023-12-21,1.010584,LARGE
2589,2023-12-21,1.020336,LARGEST


In [110]:
df_group=df.groupby('Date').agg('max')['growth_7d']

In [111]:
df2=df.merge(df_group,on ='growth_7d',how='right')

# **Answer Q3**

In [112]:
perc_outherperform= ((df2['ticket_category']=='LARGE').sum())/df2.shape[0]*100
print(f"The share of days (percentage as int) when Large Stocks outperform (on growth_7d) the Largest stocks {perc_outherperform:.0f}")

the share of days (percentage as int) when Large Stocks outperform (on growth_7d) the Largest stocks 47


# Question 4: Trying Another Technical Indicators strategy
**What's the total gross profit (in THOUSANDS of $) you'll get from trading on CCI (no fees assumption)?**

First, run the entire Colab to obtain the full DataFrame of data (after [Code Snippet 9]), and truncate it to the last full 10 years of data (2014-01-01 to 2023-12-31). If you encounter any difficulties running the Colab - you can download it using this link.

Let's assume you've learned about the awesome CCI indicator (Commodity Channel Index), and decided to use only it for your operations.

You defined the "defensive" value of a high threshould of 200, and you trade only on Fridays (Date.dt.dayofweek()==4).

That is, every time you see that CCI is >200 for any stock (out of those 33), you'll invest $1000 (each record when CCI>200) at Adj.Close price and hold it for 1 week (5 trading days) in order to sell at the Adj. Close price.

What's the expected gross profit (no fees) that you get in THOUSANDS $ (closest integer value) over many operations in 10 years? One operation calculations: if you invested $1000 and received $1010 in 5 days - you add $10 to gross profit, if you received $980 - add -$20 to gross profit. You need to sum these results over all trades (460 times in 10 years).

Additional:

Add an approximate fees calculation over the 460 trades from this calculator https://www.degiro.ie/fees/calculator (Product:"Shares, USA and Canada;" Amount per transaction: "1000 EUR"; Transactions per year: "460")
are you still profitable on those trades?

In [113]:

from google.colab import drive
drive.mount('content')

# from google.colab import files

''

Mounted at content


''

In [123]:
# !wget 'https://drive.google.com/file/d/1m3Qisfs2XfWk6Sw_Uk5kHLWqwQ0q8SKb/view?usp=sharing'

In [131]:
!ls 'content/MyDrive/DataClub/Stock_Market_Analytics_Zoomcamp'

Module01_Colab_Introduction_and_Data_Sources.ipynb
Module01_Data_Sources_and_Calculations_Assignment.ipynb
Module02_Working_With_The_Data_Assignment.ipynb
Module2_Colab_Working_with_the_data.ipynb
stocks_df_combined_trunc_2014_2023.parquet.brotli.parquet.brotli


In [135]:
path='content/MyDrive/DataClub/Stock_Market_Analytics_Zoomcamp'

In [136]:
m16 = pd.read_parquet(f"{path}/stocks_df_combined_trunc_2014_2023.parquet.brotli")

In [137]:
m16

,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Year,Month,Weekday,...,growth_brent_oil_7d,growth_brent_oil_30d,growth_brent_oil_90d,growth_brent_oil_365d,growth_btc_usd_1d,growth_btc_usd_3d,growth_btc_usd_7d,growth_btc_usd_30d,growth_btc_usd_90d,growth_btc_usd_365d
7011,37.349998,37.400002,37.099998,37.160000,31.233059,30632200.0,MSFT,2014,2014-01-01,3,...,0.964302,0.992998,0.970030,1.158676,NaN,NaN,NaN,NaN,NaN,NaN
7012,37.200001,37.220001,36.599998,36.910000,31.022930,31134800.0,MSFT,2014,2014-01-01,4,...,0.958139,0.984707,0.961500,1.143209,NaN,NaN,NaN,NaN,NaN,NaN
7013,36.849998,36.889999,36.110001,36.130001,30.367352,43603700.0,MSFT,2014,2014-01-01,0,...,0.953798,0.998223,0.968951,1.168236,NaN,NaN,NaN,NaN,NaN,NaN
7014,36.330002,36.490002,36.209999,36.410000,30.602673,35802800.0,MSFT,2014,2014-01-01,1,...,0.958653,0.993430,0.977598,1.097648,NaN,NaN,NaN,NaN,NaN,NaN
7015,36.000000,36.139999,35.580002,35.759998,30.056356,59971700.0,MSFT,2014,2014-01-01,2,...,0.955161,0.973383,0.974977,1.100781,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5338,3424.000000,3496.000000,3408.600098,3477.949951,3477.949951,1681707.0,LT.NS,2023,2023-12-01,4,...,1.064772,0.971018,0.939967,0.797881,1.002935,1.040865,1.049324,1.175398,1.655339,2.614201
5339,3477.949951,3508.350098,3477.949951,3490.050049,3490.050049,1072263.0,LT.NS,2023,2023-12-01,1,...,1.058217,0.982429,0.956014,0.801404,0.974945,0.972127,1.005911,1.134509,1.613511,2.513055
5340,3510.000000,3549.000000,3504.149902,3544.000000,3544.000000,1389266.0,LT.NS,2023,2023-12-01,2,...,1.040496,0.965806,0.943050,0.749506,1.021694,1.009920,0.995203,1.166121,1.607712,2.598696
5341,3545.000000,3559.949951,3500.500000,3518.050049,3518.050049,3371121.0,LT.NS,2023,2023-12-01,3,...,1.005645,0.965632,0.932881,0.730228,0.981240,0.977409,0.971705,1.126794,1.583988,2.575301


In [138]:
# m16.info()
m17= m16[['Date','Adj Close_x','Ticker','cci','Weekday']].copy()
m17['gross profit']=0
m17.reset_index(inplace=True)
m17=m17.drop(labels='index',axis=1)
m17.loc[0,'gross_profit']=1000
m17

,Date,Adj Close_x,Ticker,cci,Weekday,gross profit,gross_profit
0,2014-01-02,31.233059,MSFT,57.700615,3,0,1000.0
1,2014-01-03,31.022930,MSFT,1.373763,4,0,NaN
2,2014-01-06,30.367352,MSFT,-96.631259,0,0,NaN
3,2014-01-07,30.602673,MSFT,-83.904297,1,0,NaN
4,2014-01-08,30.056356,MSFT,-147.855135,2,0,NaN
...,...,...,...,...,...,...,...
80757,2023-12-22,3477.949951,LT.NS,70.767162,4,0,NaN
80758,2023-12-26,3490.050049,LT.NS,99.598220,1,0,NaN
80759,2023-12-27,3544.000000,LT.NS,130.401152,2,0,NaN
80760,2023-12-28,3518.050049,LT.NS,106.774509,3,0,NaN


In [139]:
# m17.info()
m17

,Date,Adj Close_x,Ticker,cci,Weekday,gross profit,gross_profit
0,2014-01-02,31.233059,MSFT,57.700615,3,0,1000.0
1,2014-01-03,31.022930,MSFT,1.373763,4,0,NaN
2,2014-01-06,30.367352,MSFT,-96.631259,0,0,NaN
3,2014-01-07,30.602673,MSFT,-83.904297,1,0,NaN
4,2014-01-08,30.056356,MSFT,-147.855135,2,0,NaN
...,...,...,...,...,...,...,...
80757,2023-12-22,3477.949951,LT.NS,70.767162,4,0,NaN
80758,2023-12-26,3490.050049,LT.NS,99.598220,1,0,NaN
80759,2023-12-27,3544.000000,LT.NS,130.401152,2,0,NaN
80760,2023-12-28,3518.050049,LT.NS,106.774509,3,0,NaN


In [140]:
m17['buy']=0
m17.loc[(m17['cci']>200) & (m17['Weekday']==4),'buy']=1
m17['buy'].sum()

460

In [141]:
lst_buy=list(m17.loc[m17['buy']==1].index)
# lst_buy

In [142]:
# on_trade= 0
# idx_trade = 0
# price_in=0
# profit = 1000
# ticker= m17.iloc[0]['Ticker']
# operation_counter = 0
# t1=time.time()


# for idx in range(1,m17.shape[0]):
# # for idx in range(1,1000):
#   if m17.iloc[idx]['buy']==1 and on_trade==0:
#     idx_trade=idx
#     on_trade = 1
#     price_in =m17.iloc[idx]['Adj Close_x']
#     ticker=m17.iloc[idx]['Ticker']
#   elif on_trade ==1 and idx_trade+5==idx  and m17.iloc[idx]['Ticker'] == ticker:
#     price_out= m17.iloc[idx]['Adj Close_x']
#     on_trade = 0
#     profit = price_out/price_in*profit
#     # print(f" pout{price_out} pin {price_in} {profit}")
#     operation_counter+=1
#   elif (not m17.iloc[idx]['Ticker'] == ticker) and on_trade ==1:
#     price_out= m17.iloc[idx-1]['Adj Close_x']
#     on_trade = 0
#     profit = price_out/price_in*profit
#     # print(f" pout{price_out} pin {price_in} {profit}")
#     operation_counter+=1

#   m17.loc[m17.iloc[idx].name,'gross_profit']=profit

# print(f"number of operations {operation_counter} , time enlapsed {time.time()-t1:.2f}")

In [143]:
on_trade= 0
idx_trade = 0
price_in=0
profit = 1000
ticker= m17.iloc[0]['Ticker']
operation_counter = 0
t1=time.time()
m17['gross_profit2']=np.NaN

for idx in lst_buy:
  idx_trade=idx
  on_trade = 1
  price_in =m17.iloc[idx]['Adj Close_x']
  ticker=m17.iloc[idx]['Ticker']
  # print(idx,profit)
  for jdx in range(idx,m17.shape[0]):
    if on_trade ==1 and idx_trade+5==jdx  and m17.iloc[jdx]['Ticker'] == ticker:
      price_out= m17.iloc[jdx]['Adj Close_x']
      on_trade = 0
      profit = price_out/price_in*profit
      operation_counter+=1
      m17.loc[m17.iloc[jdx].name,'gross_profit2']=profit
      break
    elif (not m17.iloc[jdx]['Ticker'] == ticker) and on_trade ==1:
      price_out= m17.iloc[jdx-1]['Adj Close_x']
      on_trade = 0
      profit = price_out/price_in*profit
      # print(f" pout{price_out} pin {price_in} {profit}")
      operation_counter+=1
      m17.loc[m17.iloc[jdx].name,'gross_profit2']=profit
      break
  # m17.loc[m17.iloc[idx].name,'gross_profit']=profit

# **Answer Q4**

In [145]:
print(f"The number of operations is {operation_counter}, the time enlapsed is {time.time()-t1:.2f}, and the profit final {profit/1000:.0f}$ thousand ")

The number of operations is 460, the time enlapsed is 50.16, and the profit final 2$ thousand 


In [146]:
# m17.fillna(method="ffill",inplace=True)

In [147]:
# m17.to_excel('test.xlsx')

In [148]:
# m17.tail()

# [EXPLORATORY] Question 5: Finding Your Strategy for IPOs
You've seen in the first questions that the median and average investments are negative in IPOs, and you can't blindly invest in all deals.

How would you correct/refine the approach? Briefly describe the steps and the data you'll try to get (it should be generally feasible to do it from public sources - no access to internal data of companies)?

E.g. (some ideas) Do you want to focus on the specific vertical? Do you want to build a smart comparison vs. existing stocks on the market? Or you just will want to get some features (which features?) like total number of people in a company to find a segment of "successful" IPOs?


To refine IPO investment strategies and address negative median and average investments:

Vertical Focus: Target specific industries with positive post-IPO performance by analyzing historical data.

Comparison with Existing Stocks: Compare financial metrics like revenue growth, profit margins, and market share to existing market stocks.

Financial Health Metrics: Consider debt-to-equity ratios, cash flow trends, and earnings stability for IPO companies.

Management Evaluation: Assess management teams' experience and success, favoring reputable leaders.

Market Trends: Align IPO selections with growing market demands and emerging trends.

Post-IPO Performance: Analyze stock price growth, market capitalization, and investor sentiment post-IPO.

Regulatory Compliance: Prioritize companies with strong legal compliance and transparent governance.